# Notebook for presentation 2

### Packages and Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn import svm as SVM
from sklearn.naive_bayes import GaussianNB as NB
from xgboost import XGBClassifier

In [2]:
# custom imports
from funcs import plot_cv_confidence_vs_profit, score_dmc_profit,dmc_profit,cv_preds_and_confusion_matrix,cv_profits_for_models, profit_scoring
from customClassifiers import CustomModelWithThreshold, TrustHard, PerceptronLearner
from pipes import CustomAttributeAdder,Scaling,RandomAttributeAdder,Transformer,ClfSwitcher

from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import VotingClassifier

In [3]:
# use sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import itertools

# Load the Data

In [4]:
# load the data
X_train = pd.read_csv('train.csv' ,delimiter="|")
X_test = pd.read_csv('test.csv', delimiter="|")
X_train, y_train = X_train.drop(columns='fraud'), X_train['fraud']
#y_test = test.pop('fraud')

In [5]:
X_train.head(1)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.7,7,0,3,0.027514,0.051898,0.241379


# Preprocess Data

In [6]:
# select one or more out of feature list below that will be added in the featureGenerationPipeline
feature_list = ['scannedLineItemsTotal',
                #'valuePerLineItem',
                #'quantityModificationsPerLineItem',
                'lineItemVoids*scansWithoutRegistration',
                'totalScanTimeInSeconds/trustLevel',
                'trustLevel_Log', 
               ]

**Note:** Only two preprocessing steps at the moment are adding newly designed features (see above) and scaling

In [7]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder(featurelist=feature_list)),                
    #("RandomAttributeAdder", RandomAttributeAdder())         #  This class is still void
    ])


preprocessing_pipeline = Pipeline([
    #("transformer", Transformer()),                           # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])

In [8]:
# combine two pipeline into a single data_preparation_pipeline
data_preparation_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline)
])

X_train_prepared = data_preparation_pipeline.fit_transform(X_train)

/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/roman/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
# Note that X_train has also been changed by calling fit_transform on it
# X_train can be used to visualize the features importance / correlation of the newly created features
X_train.head(5)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,scannedLineItemsTotal,lineItemVoids*scansWithoutRegistration,totalScanTimeInSeconds/trustLevel,trustLevel_Log
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,29.0,0,210.800000,1.609438
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,14.0,10,36.000000,1.098612
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,13.0,30,505.333333,1.098612
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,29.0,32,298.500000,1.791759
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,27.0,21,86.000000,1.609438


### Correlation

In [ ]:
#...

# Parameter Tuning

### Evaluation 

In [10]:
# Nico's script
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer

cv = StratifiedKFold(n_splits=10, random_state=42)
def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))
profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

## 1. Tune Hyperparameter of Base Classifiers (without using sampling)

In [11]:
# Install the library for Bayesian optimization from here: https://github.com/fmfn/BayesianOptimization
from bayes_opt import BayesianOptimization

In [12]:
X_train_prepared.shape

(1879, 13)

### 1.1 SGD Classifier

In [13]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol, penalty=penalty_str, loss=loss_str, random_state=231)
    
    # This integrates sampling into the training. Trains on oversampled data but evaluates on unsampled data
    #return cross_val_imbalanced(model, X_train_prepared, y_train, RandomOverSampler(random_state=42))
    
    # this trains the classifier on the unbalanced folds
    return sum(cross_validate(model, X_train_prepared, y=y_train, cv=cv,
                              scoring=profit_scoring)['test_score'])

In [14]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [15]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=200, init_points=20)

|   iter    |  target   |   alpha   | l1_ratio  |   loss    |  penalty  |    tol    |
-------------------------------------------------------------------------------------
|  1        | -520.0    |  0.7815   |  0.5842   |  1.28     |  0.946    |  0.08269  |
|  2        | -490.0    |  0.9034   |  0.03894  |  2.746    |  0.1815   |  0.01807  |
|  3        |  75.0     |  0.0723   |  0.4588   |  2.125    |  1.45     |  0.002545 |
|  4        | -460.0    |  0.0669   |  0.4259   |  0.6577   |  2.43     |  0.01401  |
|  5        | -520.0    |  0.7512   |  0.2791   |  2.847    |  2.655    |  0.06102  |
|  6        | -520.0    |  0.7791   |  0.04156  |  2.359    |  2.707    |  0.0433   |
|  7        | -520.0    |  0.8866   |  0.5485   |  2.961    |  1.696    |  0.0901   |
|  8        | -520.0    |  0.5637   |  0.0959   |  1.853    |  0.18     |  0.05804  |
|  9        | -520.0    |  0.8717   |  0.7998   |  0.2      |  0.1805   |  0.08993  |
|  10       | -520.0    |  0.6437   |  0.668    |  0.3

|  58       |  130.0    |  0.04729  |  0.4817   |  2.173    |  1.4      |  0.003144 |
|  59       |  95.0     |  0.06657  |  0.4546   |  2.137    |  1.457    |  0.007613 |
|  60       |  200.0    |  0.01015  |  0.5133   |  2.089    |  1.537    |  0.06189  |
|  61       |  210.0    |  0.006535 |  0.5301   |  2.075    |  1.543    |  0.05408  |
|  62       |  225.0    |  0.008352 |  0.5273   |  2.087    |  1.545    |  0.05553  |
|  63       |  55.0     |  0.05994  |  0.405    |  2.082    |  1.507    |  0.04087  |
|  64       |  170.0    |  0.02173  |  0.521    |  2.118    |  1.498    |  0.05299  |
|  65       |  165.0    |  0.01101  |  0.5108   |  2.142    |  1.418    |  0.05881  |
|  66       |  120.0    |  0.05528  |  0.4722   |  2.141    |  1.496    |  0.0975   |
|  67       |  120.0    |  0.04856  |  0.4853   |  2.155    |  1.445    |  0.02392  |
|  68       |  150.0    |  0.03434  |  0.524    |  2.099    |  1.516    |  0.09977  |
|  69       |  165.0    |  0.04077  |  0.5201   |  2.1

|  116      |  180.0    |  0.01673  |  0.5409   |  2.137    |  1.515    |  0.08684  |
|  117      |  75.0     |  0.04798  |  0.3992   |  2.113    |  1.535    |  0.04553  |
|  118      |  70.0     |  0.06173  |  0.449    |  2.153    |  1.468    |  0.01573  |
|  119      |  140.0    |  0.03584  |  0.4807   |  2.13     |  1.464    |  0.05243  |
|  120      |  5.0      |  0.07875  |  0.4181   |  2.116    |  1.522    |  0.0609   |
|  121      | -25.0     |  0.07077  |  0.4904   |  2.126    |  1.461    |  0.05771  |
|  122      |  115.0    |  0.03812  |  0.4073   |  2.096    |  1.479    |  0.09047  |
|  123      |  200.0    |  0.009539 |  0.5117   |  2.166    |  1.374    |  0.06377  |
|  124      |  110.0    |  0.04867  |  0.5119   |  2.148    |  1.442    |  0.02441  |
|  125      |  210.0    |  0.02219  |  0.4994   |  2.121    |  1.474    |  0.01561  |
|  126      |  220.0    |  0.004471 |  0.5212   |  2.068    |  1.578    |  0.08904  |
|  127      |  140.0    |  0.03541  |  0.4825   |  2.1

|  174      |  115.0    |  0.0522   |  0.5547   |  2.181    |  1.372    |  0.03058  |
|  175      |  105.0    |  0.04373  |  0.4352   |  2.076    |  1.49     |  0.02847  |
|  176      |  110.0    |  0.05061  |  0.4586   |  2.129    |  1.451    |  0.0213   |
|  177      |  170.0    |  0.02161  |  0.5107   |  2.132    |  1.515    |  0.08394  |
|  178      | -520.0    |  0.9489   |  0.6248   |  1.05     |  1.504    |  0.07091  |
|  179      | -35.0     |  0.06448  |  0.5685   |  2.169    |  1.373    |  0.02517  |
|  180      |  100.0    |  0.05576  |  0.499    |  2.057    |  1.538    |  0.09896  |
|  181      |  110.0    |  0.05185  |  0.4927   |  2.164    |  1.433    |  0.02433  |
|  182      |  155.0    |  0.04437  |  0.5131   |  2.24     |  1.417    |  0.06134  |
|  183      | -520.0    |  0.9694   |  0.5027   |  2.017    |  2.299    |  0.03359  |
|  184      |  90.0     |  0.05153  |  0.5103   |  2.232    |  1.392    |  0.02393  |
|  185      |  260.0    |  0.004259 |  0.4571   |  2.1

In [16]:
optimization_sgd.max

{'target': 260.0,
 'params': {'alpha': 0.005869942773702552,
  'l1_ratio': 0.4650232398687586,
  'loss': 2.124838792024528,
  'penalty': 1.4489392051726289,
  'tol': 0.04443900510956935}}

### 1.2 Logistic Regression

In [17]:
def evaluateLogistic(C):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42)
    
    return sum(cross_validate(model, X_train_prepared, y=y_train, cv=cv,
                   scoring=profit_scoring)['test_score'])

In [18]:
params_logistic = {
    'C': (1,1000),
}

In [19]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=231)
optimization_logistic.maximize(n_iter=200, init_points=20)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  280.0    |  781.7    |
|  2        |  305.0    |  584.6    |
|  3        |  295.0    |  427.3    |
|  4        |  295.0    |  316.0    |
|  5        |  280.0    |  827.1    |
|  6        |  280.0    |  903.5    |
|  7        |  335.0    |  39.9     |
|  8        |  280.0    |  915.3    |
|  9        |  310.0    |  61.44    |
|  10       |  320.0    |  181.5    |
|  11       |  310.0    |  73.23    |
|  12       |  295.0    |  459.3    |
|  13       |  280.0    |  708.7    |
|  14       |  305.0    |  483.8    |
|  15       |  335.0    |  26.42    |
|  16       |  310.0    |  67.84    |
|  17       |  295.0    |  426.5    |
|  18       |  320.0    |  220.0    |
|  19       |  280.0    |  810.3    |
|  20       |  320.0    |  140.9    |
|  21       |  335.0    |  33.47    |
|  22       |  335.0    |  37.44    |
|  23       |  335.0    |  29.22    |
|  24       |  335.0    |  38.89    |
|  25       

|  132      |  335.0    |  49.61    |
|  133      |  310.0    |  50.02    |
|  134      |  335.0    |  49.45    |
|  135      |  335.0    |  49.03    |
|  136      |  335.0    |  49.55    |
|  137      |  335.0    |  25.98    |
|  138      |  335.0    |  28.64    |
|  139      |  335.0    |  33.7     |
|  140      |  335.0    |  20.0     |
|  141      |  335.0    |  48.23    |
|  142      |  335.0    |  24.72    |
|  143      |  335.0    |  21.06    |
|  144      |  335.0    |  47.46    |
|  145      |  335.0    |  18.01    |
|  146      |  335.0    |  38.69    |
|  147      |  335.0    |  46.72    |
|  148      |  335.0    |  46.0     |
|  149      |  335.0    |  37.25    |
|  150      |  335.0    |  28.24    |
|  151      |  335.0    |  21.45    |
|  152      |  335.0    |  45.29    |
|  153      |  335.0    |  38.29    |
|  154      |  335.0    |  29.03    |
|  155      |  335.0    |  44.6     |
|  156      |  335.0    |  48.62    |
|  157      |  335.0    |  43.93    |
|  158      

In [28]:
optimization_logistic.max

{'target': 335.0, 'params': {'C': 39.89916730939222}}

### 1.3 SVM

In [20]:
#...

### 1.4 RandomForest

In [21]:
#...

### 1.5 XGBoost

In [22]:
#...

### 1.6 AdaBoost

In [29]:
#...

### Bagging, Stacking, Voting, etc..

In [32]:
#...

## 2. Tune Hyperparameter of Base Classifiers (inlc. sampling)

In [33]:
# import sampling classes

# oversampling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE

# undersampling
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.under_sampling import InstanceHardnessThreshold
from imblearn.under_sampling import TomekLinks

# combination of over and undersampling
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [34]:
# Note use this function is used to evaluate sampling + cross validation!
def cross_val_imbalanced(classifier, X, y, sampler):
    kf = StratifiedKFold(n_splits=10, random_state=42)
    cross_val_profit_lst = []
    
    X = pd.DataFrame(X)

    for train_index, test_index in kf.split(X, y):
        # keeping validation set apart and oversampling in each iteration using smote 
        train, test = X.iloc[train_index], X.iloc[test_index]
        target_train, target_test = y.iloc[train_index], y.iloc[test_index]
        X_train_res, y_train_res = sampler.fit_sample(train, target_train.ravel())

        # training the model on oversampled 9 folds of training set
        classifier.fit(pd.DataFrame(X_train_res), y_train_res)
        # testing on 1 fold of validation set
        test_preds = classifier.predict(test)
        cross_val_profit_lst.append(profit_scorer(target_test, test_preds))
        
    return np.sum(cross_val_profit_lst)

### 2.1 RandomOverSampling + SGD Classifier

In [35]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol,
                          penalty=penalty_str, loss=loss_str, random_state=231,
                         n_jobs=-1)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, RandomOverSampler(random_state=42))
    
    #return sum(cross_validate(model, X_train_prepared, y=y_train, cv=cv,
                   #scoring=profit_scoring)['test_score'])

In [36]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [37]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=200, init_points=20)

|   iter    |  target   |   alpha   | l1_ratio  |   loss    |  penalty  |    tol    |
-------------------------------------------------------------------------------------
|  1        | -5.655e+0 |  0.7815   |  0.5842   |  1.28     |  0.946    |  0.08269  |
|  2        | -3.605e+0 |  0.9034   |  0.03894  |  2.746    |  0.1815   |  0.01807  |
|  3        | -480.0    |  0.0723   |  0.4588   |  2.125    |  1.45     |  0.002545 |
|  4        | -2.885e+0 |  0.0669   |  0.4259   |  0.6577   |  2.43     |  0.01401  |
|  5        | -4.975e+0 |  0.7512   |  0.2791   |  2.847    |  2.655    |  0.06102  |
|  6        | -5.1e+03  |  0.7791   |  0.04156  |  2.359    |  2.707    |  0.0433   |
|  7        | -4.315e+0 |  0.8866   |  0.5485   |  2.961    |  1.696    |  0.0901   |
|  8        | -4.98e+03 |  0.5637   |  0.0959   |  1.853    |  0.18     |  0.05804  |
|  9        | -5.68e+03 |  0.8717   |  0.7998   |  0.2      |  0.1805   |  0.08993  |
|  10       | -5.7e+03  |  0.6437   |  0.668    |  0.3

|  58       | -130.0    |  0.04729  |  0.4817   |  2.173    |  1.4      |  0.003144 |
|  59       | -465.0    |  0.07584  |  0.4617   |  2.138    |  1.449    |  0.006097 |
|  60       | -755.0    |  0.07852  |  0.9549   |  2.681    |  0.8793   |  0.08541  |
|  61       |  155.0    |  0.00266  |  0.4962   |  2.173    |  1.417    |  0.03833  |
|  62       | -705.0    |  0.07468  |  0.9893   |  2.657    |  0.8706   |  0.05708  |
|  63       | -275.0    |  0.08822  |  0.5178   |  2.126    |  1.422    |  0.004752 |
|  64       | -575.0    |  0.1011   |  0.466    |  2.171    |  1.416    |  0.02991  |
|  65       |  120.0    |  0.01101  |  0.5108   |  2.142    |  1.418    |  0.05881  |
|  66       | -830.0    |  0.08497  |  0.9612   |  2.691    |  0.7991   |  0.07223  |
|  67       | -400.0    |  0.08514  |  0.4535   |  2.124    |  1.405    |  0.05365  |
|  68       | -350.0    |  0.09636  |  0.4529   |  2.093    |  1.449    |  0.006357 |
|  69       | -605.0    |  0.0624   |  0.9463   |  2.6

|  116      |  75.0     |  0.03049  |  0.4774   |  2.185    |  1.394    |  0.02224  |
|  117      | -830.0    |  0.08645  |  0.9738   |  2.652    |  0.8549   |  0.03615  |
|  118      | -225.0    |  0.08813  |  0.4819   |  2.17     |  1.411    |  0.01065  |
|  119      | -25.0     |  0.03584  |  0.4807   |  2.13     |  1.464    |  0.05243  |
|  120      | -365.0    |  0.07907  |  0.4615   |  2.1      |  1.418    |  0.04586  |
|  121      | -315.0    |  0.07077  |  0.4904   |  2.126    |  1.461    |  0.05771  |
|  122      | -205.0    |  0.05088  |  0.4599   |  2.179    |  1.405    |  0.05159  |
|  123      | -255.0    |  0.0556   |  0.4962   |  2.149    |  1.436    |  0.07352  |
|  124      | -115.0    |  0.04867  |  0.5119   |  2.148    |  1.442    |  0.02441  |
|  125      | -340.0    |  0.07382  |  0.4631   |  2.242    |  1.49     |  0.03791  |
|  126      | -300.0    |  0.08695  |  0.4822   |  2.17     |  1.411    |  0.008361 |
|  127      |  0.0      |  0.03541  |  0.4825   |  2.1

|  174      | -280.0    |  0.05741  |  0.4467   |  2.116    |  1.403    |  0.03389  |
|  175      | -600.0    |  0.1011   |  0.4952   |  2.202    |  1.4      |  0.02511  |
|  176      | -180.0    |  0.05061  |  0.4586   |  2.129    |  1.451    |  0.0213   |
|  177      | -430.0    |  0.06643  |  0.4836   |  2.119    |  1.431    |  0.003301 |
|  178      |  145.0    |  0.001408 |  0.4322   |  2.18     |  1.38     |  0.02226  |
|  179      | -600.0    |  0.1045   |  0.4898   |  2.239    |  1.444    |  0.03373  |
|  180      | -365.0    |  0.07745  |  0.4614   |  2.245    |  1.487    |  0.03729  |
|  181      | -155.0    |  0.05185  |  0.4927   |  2.164    |  1.433    |  0.02433  |
|  182      | -600.0    |  0.107    |  0.5087   |  2.223    |  1.391    |  0.01214  |
|  183      | -405.0    |  0.06515  |  0.4651   |  2.228    |  1.531    |  0.02417  |
|  184      | -150.0    |  0.05153  |  0.5103   |  2.232    |  1.392    |  0.02393  |
|  185      |  195.0    |  0.004259 |  0.4571   |  2.1

In [39]:
optimization_sgd.max

{'target': 195.0,
 'params': {'alpha': 0.004259170002503944,
  'l1_ratio': 0.4571157730347052,
  'loss': 2.1384069293948227,
  'penalty': 1.4594434596038075,
  'tol': 0.038444190700582276}}

### 2.2 BorderlineSMOTE + LogisticRegression

In [52]:
def evaluateLogistic(C, k_neighbors, m_neighbors):
    
    model = LogisticRegression(C=C, solver='liblinear', random_state=42,)
    
    sampler = BorderlineSMOTE(random_state=42, k_neighbors=int(k_neighbors), m_neighbors=int(m_neighbors))

    return cross_val_imbalanced(model, X_train_prepared, y_train, sampler)

In [53]:
params_logistic = {
    'C': (1,500),
    'k_neighbors': (2, 15),
    'm_neighbors': (2, 15),
}

In [54]:
optimization_logistic = BayesianOptimization(evaluateLogistic, params_logistic, random_state=42)
optimization_logistic.maximize(n_iter=150, init_points=50)

|   iter    |  target   |     C     | k_neig... | m_neig... |
-------------------------------------------------------------
|  1        |  125.0    |  187.9    |  14.36    |  11.52    |
|  2        |  110.0    |  299.7    |  4.028    |  4.028    |
|  3        | -65.0     |  29.98    |  13.26    |  9.814    |
|  4        |  180.0    |  354.3    |  2.268    |  14.61    |
|  5        |  110.0    |  416.4    |  4.76     |  4.364    |
|  6        | -25.0     |  92.52    |  5.955    |  8.822    |
|  7        |  50.0     |  216.5    |  5.786    |  9.954    |
|  8        |  0.0      |  70.61    |  5.798    |  6.763    |
|  9        |  75.0     |  228.6    |  12.21    |  4.596    |
|  10       |  100.0    |  257.6    |  9.701    |  2.604    |
|  11       |  110.0    |  304.2    |  4.217    |  2.846    |
|  12       |  165.0    |  474.5    |  14.55    |  12.51    |
|  13       |  25.0     |  153.0    |  3.27     |  10.9     |
|  14       |  75.0     |  220.6    |  3.586    |  8.437    |
|  15   

|  81       |  180.0    |  252.5    |  2.87     |  5.794    |
|  82       |  125.0    |  182.4    |  14.71    |  14.63    |
|  83       |  125.0    |  409.3    |  13.2     |  2.043    |
|  84       |  125.0    |  182.5    |  14.7     |  14.67    |
|  85       |  205.0    |  251.5    |  2.342    |  4.65     |
|  86       |  205.0    |  251.7    |  2.866    |  5.619    |
|  87       |  205.0    |  251.9    |  2.839    |  5.999    |
|  88       |  180.0    |  252.1    |  2.741    |  5.556    |
|  89       |  205.0    |  251.0    |  2.355    |  5.915    |
|  90       |  205.0    |  252.4    |  2.411    |  6.028    |
|  91       |  205.0    |  252.5    |  2.738    |  5.657    |
|  92       |  205.0    |  251.7    |  2.29     |  5.756    |
|  93       |  205.0    |  251.6    |  2.698    |  5.819    |
|  94       |  205.0    |  251.2    |  2.13     |  5.935    |
|  95       |  205.0    |  251.1    |  2.428    |  5.943    |
|  96       |  205.0    |  251.3    |  2.459    |  5.794    |
|  97   

|  162      |  205.0    |  252.7    |  2.353    |  5.901    |
|  163      |  205.0    |  251.5    |  2.295    |  4.684    |
|  164      |  205.0    |  251.8    |  2.056    |  5.49     |
|  165      |  205.0    |  252.6    |  2.34     |  6.02     |
|  166      |  205.0    |  252.3    |  2.851    |  5.578    |
|  167      |  205.0    |  251.8    |  2.74     |  6.202    |
|  168      |  205.0    |  251.2    |  2.158    |  5.889    |
|  169      |  205.0    |  251.8    |  2.926    |  6.06     |
|  170      |  180.0    |  251.6    |  2.664    |  5.494    |
|  171      |  205.0    |  354.1    |  2.285    |  14.63    |
|  172      |  205.0    |  251.4    |  2.09     |  5.83     |
|  173      |  205.0    |  252.7    |  2.581    |  5.645    |
|  174      |  205.0    |  250.7    |  2.503    |  6.145    |
|  175      |  205.0    |  251.4    |  2.835    |  5.433    |
|  176      |  205.0    |  252.3    |  2.682    |  5.697    |
|  177      |  205.0    |  252.7    |  2.759    |  5.645    |
|  178  

In [55]:
optimization_logistic.max

{'target': 205.0,
 'params': {'C': 251.83683259120187,
  'k_neighbors': 2.6692237662498615,
  'm_neighbors': 5.622404035075949}}

In [ ]:
#...